## ДЗ 3

Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы



In [18]:
import pandas as pd
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [19]:
df_pos = pd.read_csv('positive.csv', sep=';', header=None, usecols=[3])
df_neg = pd.read_csv('negative.csv', sep=';', header=None, usecols=[3])
df = pd.concat([df_neg, df_pos])
df.columns=['text']
df

,text
0,на работе был полный пиддес :| и так каждое за...
1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,@elina_4post как говорят обещаного три года жд...
3,"Желаю хорошего полёта и удачной посадки,я буду..."
4,"Обновил за каким-то лешим surf, теперь не рабо..."
...,...
114906,"Спала в родительском доме, на своей кровати......"
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...
114908,"Что происходит со мной, когда в эфире #proacti..."
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како..."


In [30]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls


sentences = [ preprocess_txt(t) for t in tqdm_notebook(df['text']) ]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/226834 [00:00<?, ?it/s]

In [85]:
def get_sentence_vector(model, sentence):
    r  = [model.wv[w] for w in sentence if w in model.wv ]
    return sum(r) / len(r) if r else np.zeros(300)
    

def build_index(model):
    index = annoy.AnnoyIndex(300 ,'angular')
    for i, snt in tqdm_notebook(enumerate(sentences)):
        v = get_sentence_vector(model, snt)
        index.add_item(i, v)
    index.build(10)
    return index
            
    
def find_nearest_tweets(model, index, tw, n=5):
    tw = preprocess_txt(tw)
    twv = get_sentence_vector(model, tw)
    nns = index.get_nns_by_vector(twv, n)
    return [df['text'].iloc[i] for i in nns]

def test_nearest_tweets(model, index, tw, n=5):
    print('----------------------------------------------------')
    print(f"Ближайшие твиты для '{tw}':\n")
    ntws = find_nearest_tweets(model, index, tw, n)
    for tw in ntws:
        print(tw, '\n')
    

In [81]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)
indexW2V = build_index(modelW2V)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

In [86]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelW2V, indexW2V, df['text'].iloc[i])


----------------------------------------------------
Ближайшие твиты для '@tancha1392, вы назвали попугая так как я хотела назвать свою дочь. Ну вот..еще одно имя вычеркиваем)':

@tancha1392, вы назвали попугая так как я хотела назвать свою дочь. Ну вот..еще одно имя вычеркиваем) 

как бы своего сына назвал?
Федя:-D 
а блезнезов как бы?
филипп и федя:-D 
#во,я,ржала:-D 

@shilderex охотник для чего существует со своими заклинаниями на снятие всех секретов?) спелл рога для чего существует? 

@GrandiSognatore не, мне четко ясно сказали, что человек якобы стесняется своего роста) 

Ну, хорошо - выбери на свой вкус. *^_^* #ff #follow 

----------------------------------------------------
Ближайшие твиты для 'RT @Kosmicheskii: бляяяяя как же мне хуёво((( отъебитесь от меня ёбаные болезни, хули прилепились((((':

RT @Kosmicheskii: бляяяяя как же мне хуёво((( отъебитесь от меня ёбаные болезни, хули прилепились(((( 

RT @fenuxizavax: а вот теперь МЮ пиздец. Сурикат на поле ;) 

RT @smilevika01

In [83]:
modelFT = FastText(sentences=sentences, vector_size=300, window=5, min_count=1)
indexFT = build_index(modelFT)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

In [87]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelFT, indexFT, df['text'].iloc[i])

----------------------------------------------------
Ближайшие твиты для '@__Stolichnaya__ ну это должно быть в первую очередь)':

@__Stolichnaya__ ну это должно быть в первую очередь) 

В человеке все должно быть прекрасно,какарда,погоны, а иначе это не человек а млекопитающее))))))))))))))))))))))))))))) 

@kalenochka но сегодня это ебаный вторник(( 

@tralfamadority все должно быть в комплексе(( так что не нааааадо тут! 

@Nataliasinger @m_grigorenko солидарен, практика полезней теории) 

----------------------------------------------------
Ближайшие твиты для 'как всё таки приятно видить радость на лице человека после того как с чем то его поздровляешь)) Скорей бы новый год!':

как всё таки приятно видить радость на лице человека после того как с чем то его поздровляешь)) Скорей бы новый год! 

@caliwinter Всё это так парадоксально:с каждым новым днём вроде и растут надежды на лучший исход,и в то же время маразм власти крепчает:( 

Расстроилась!!! Огорченна! Разочарованна!
Не хвата

In [92]:
qs = ['как дела', 'какая замечательная погода']
for q in qs:
    test_nearest_tweets(modelW2V, indexW2V, q, 3)

----------------------------------------------------
Ближайшие твиты для 'как дела':

Вот будто другого нет, что я в самом деле:(( 

@shiki__24 да(( но одно дело там, а другое дело уже что то тут, снова...бляблябля 

если бы было так на самом деле((((((((((((((( http://t.co/kjeEzWwD63 

----------------------------------------------------
Ближайшие твиты для 'какая замечательная погода':

Не радует погода меня(((( http://t.co/r2CJ0G2pB8 

Погода прекрасна, настрій піднесений, життя чудове!;) 

Психодел-фото) но погода обалденная!) несмотря на всю мою мерзлячность))) @ Международный аэропорт… http://t.co/n4KlcIkQxj 



In [93]:
for q in qs:
    test_nearest_tweets(modelFT, indexFT, q, 3)

----------------------------------------------------
Ближайшие твиты для 'как дела':

Вот будто другого нет, что я в самом деле:(( 

Сегодня я не буду с вами, ибо много дел :( 

@annanekipelova4 не смогла, дел много было( 

----------------------------------------------------
Ближайшие твиты для 'какая замечательная погода':

Погода просто ужас ((( А ведь всего через несколько дней Новый Год! 

День весьма необычный, но погода не айс: ( 

Поскорее бы Новый Год, будет весело, ждуууу ;) ;) ;) 



Сравнила word2vec и fasttext. 2-3 ответа на вопросы получились адекватные в обоих случаях, совпадение твитов - не очень, кажется, что FastText работает чуть лучше, но объективной метрики нет. 